<a href="https://colab.research.google.com/github/chldydgh4687/PRproject/blob/Hotfix/re_spm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# * Kaggle Connecting

In [1]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

-rw-r--r-- 1 root root 66 Dec  4 05:19 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=53b5f63557e6c7fd98a2421ae8e2453899e37738b19fe996307d94a9fc25bfc8
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Kaggle API 1.5.6


In [2]:
! kaggle competitions download -c 2019-ml-finalproject

 63% 41.0M/65.0M [00:00<00:00, 59.4MB/s]
100% 65.0M/65.0M [00:00<00:00, 152MB/s] 


In [0]:
import zipfile
import os

os.mkdir('/content/input')

zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

# * opencv version changing

In [4]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.7.28:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.7.28.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtCore-9549151f.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtGui-6d0f14dd.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavcodec-eac15e48.so.58.21.104
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavformat-b6bcbe33.so.58.17.101
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavutil-e1b1a17d.so.56.18.102
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswresample-b4363bfa.so.3.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswscale-15b3fdc6.so.5.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libvpx-81a43c0a.so.5.0.0
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.7.28
Uninstalling opencv-contrib-python-3.4.3.

# * import library

In [0]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torch
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook as tqdm

import glob

In [6]:
cv2.__version__

'3.4.2'

# * Feature Extraction
- Dense Sift

In [0]:
# label readying.
df_data = pd.read_csv('/content/input/Label2Names.csv', header=None)

In [30]:
# train dense sift
train_path = "/content/input/train"

train_des = list()
train_images = list()
train_label = list()
train_fea = list()

dense = 16
img_size = 256

for cls in tqdm(os.listdir(train_path)):
  img_list = os.listdir(train_path + '/' + cls)
  img_list.sort()
  if cls == 'BACKGROUND_Google':
    label = 102
  else :
    label = (df_data.index[df_data[1]==cls] + 1).tolist()[0]

  for img in img_list:

    image = cv2.imread(train_path + '/' + cls + '/' + img)

    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (img_size, img_size))
    sift = cv2.xfeatures2d.SIFT_create()

    kp = [cv2.KeyPoint(x, y, dense) for y in range(0, gray.shape[0]
                                                   , dense) for x in range(0, gray.shape[1], dense)]
    keypoint, des = sift.compute(gray, kp)

    train_des.append(des)
    train_fea.append(des)
    train_images.append(gray)
    train_label.append(label)

In [23]:
# test dense sift
test_path = "/content/input/testAll_v2"

test_des = list()
test_images = list()
test_fea = list()

dense = 16
img_size = 256

img_list = os.listdir(test_path)
img_list.sort()

for img in tqdm(img_list):
  image = cv2.imread(test_path + '/' + img)

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (img_size, img_size))
  sift = cv2.xfeatures2d.SIFT_create()

  kp = [cv2.KeyPoint(x, y, dense) for y in range(0, gray.shape[0]
                                                   , dense) for x in range(0, gray.shape[1], dense)]
  keypoint, des = sift.compute(gray, kp)
  test_des.append(des)
  test_images.append(gray)
  test_fea.append(des)

In [0]:
# import pickle

# # Save pickle
# with open("densetrain_des.pickle", "wb") as f:
#   pickle.dump(train_des, f)

# with open("densetest_des.pickle", "wb") as f:
#   pickle.dump(test_des, f)


In [0]:
# with open("densetrain_des.pickle", "rb") as r:
#   read_data = pickle.load(r)

# with open("densetest_des.pickle", "rb") as r:
#   read_data = pickle.load(r)

In [34]:
train_images = np.array(train_images)
train_images.shape

(3060, 256, 256)

In [33]:
train_fea = np.array(train_fea)
train_fea.shape

(3060, 256, 128)

# * Checking the shape

In [12]:
train_des = np.array(train_des)
train_des.shape

(3060, 256, 128)

In [21]:
test_des = np.array(test_des)
test_des.shape

(1692, 256, 128)

In [13]:
# for codebook histogram, reshape 1d, 128 dimensions

train_des = train_des.reshape(-1,128)
train_des.shape

(783360, 128)

In [22]:
test_des = test_des.reshape(-1,128)
test_des.shape

(433152, 128)

# * 0 depth.ver (Before SPM) >= 0.41 

In [14]:
# gpu kmeans install
pip install kmc2

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.8MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252217 sha256=2b315f384910d44ad853714d05ffffa2efff40545c06fa6c542aa2abfd0ac7ad
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import kmc2
import copy
from sklearn.cluster import MiniBatchKMeans

In [0]:
# making set histograms

def making_histograms(train_data, kmeans, k):
  data_hist = []
  for i in range(len(train_data)):
    data = copy.deepcopy(train_data[i])
    predict = kmeans.predict(data)
    data_hist.append(np.bincount(predict, minlength = k).reshape(1,-1).ravel())
    
  return np.array(data_hist)

In [18]:
#특징에 대한 클러스터링을 진행

seeding = kmc2.kmc2(train_des, 200)
kmeans = MiniBatchKMeans(200, init = seeding).fit(train_des)
codebook = kmeans.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# 한번더 뿌려서 clustering 시켜서 codebook 생성
train_hist = making_histograms(train_fea, kmeans, 200)

In [0]:
test_hist = making_histograms(test_fea, kmeans, 200)

In [0]:
# spm kernel

def histogramIntersection(M,N):
  m = M.shape[0]
  n = N.shape[0]
  result = np.zeros ((m,n))
  for i in range(m):
    for j in range(n):
      temp = np.sum(np.minimum(M[i],N[j]))
      result[i][j] = temp
  return result

In [0]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA

from sklearn.pipeline import make_pipeline

In [0]:
gramMatrix = histogramIntersection(train_hist,train_hist)
clf = SVC(kernel='precomputed')
clf.fit(gramMatrix, train_label)

predictMatrix = histogramIntersection(test_hist,train_hist)
SVMresults = clf.predict(predictMatrix)

In [46]:
SVMresults.shape

(1692,)

In [0]:
img_list = os.listdir("/content/input/testAll_v2")
img_list.sort()

In [0]:
yfit = SVMresults.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,yfit])

In [0]:
df = pd.DataFrame(total_result, columns=["Id","Category"])
df.to_csv('results-yongho-v3.csv',index=False, header=True)

In [50]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-yongho-v3.csv -m "Final_Term_Project"

100% 29.6k/29.6k [00:00<00:00, 114kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 